# 환경설정

In [15]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json
WAIT = 3
#options = webdriver.ChromeOptions()
#options.add_argument('headless')

# 함수 정의

In [16]:
# css 찾을때 까지 10초대기
def time_wait(num, code,driver):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [17]:
def find_search_input(key_word,driver):
    # 검색창 찾기
    driver.switch_to.default_content()
    time_wait(2, 'div.input_box > input.input_search',driver)
    search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
    search.send_keys(key_word)  # 검색어 입력
    driver.implicitly_wait(3)
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
    sleep(5)
    res = driver.page_source  # 페이지 소스 가져오기
    soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다
    sleep(1)

In [18]:
# frame 변경 메소드
def switch_frame(frame,driver):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    driver.implicitly_wait(1)


In [19]:
# 페이지 다운
def page_down(num,driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

In [20]:
# 페이지 다운
def page_end(driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    body.send_keys(Keys.END)

In [21]:
def find_review_btn(driver):
    sleep(2)
    time_wait(10,'a.tpj9w',driver)
    menu_list = driver.find_elements(By.CSS_SELECTOR,'a.tpj9w')
    if len(menu_list) ==7:
        return menu_list[-1]
    else:
        return menu_list[-2]

In [43]:
def crawling_menu_review(driver):
    #-----리뷰 버튼 클릭-------
    find_review_btn(driver).click()
    SCROLL_PAUSE_SEC = 1
    driver.implicitly_wait(WAIT)
    #메뉴 요소 찾기
    menu_elements  = driver.find_elements(By.CSS_SELECTOR,'div.place_fixed_subtab > div > div:nth-child(1) > div > div > div > span > a> span:first-child')
    sleep(SCROLL_PAUSE_SEC)
    driver.implicitly_wait(WAIT)
    try:
        #왼쪽 버튼 찾기
        left_btn = driver.find_element(By.CSS_SELECTOR,'a.PznE8.aNw43')
        driver.implicitly_wait(WAIT)
        left_btn.click()
        driver.implicitly_wait(WAIT)
    except:
        pass

    try:
        #오른족 버튼 찾기
        right_btn = driver.find_element(By.CSS_SELECTOR, 'div.place_fixed_subtab > div > div:nth-child(1) > div > div > a.ZCqf_.aNw43')
        driver.implicitly_wait(WAIT)
    except:
        pass
    menu_list = list()
    #-----------여기까지-----------
    i = 0
    
    # menu_elentents가 존재하지 않는다면 except로 갈 것이다.
    try: 
        while True:
            if menu_elements[i].text != '': #요소가 존재하고 메뉴 텍스트도 보일때
                menu_list.append(menu_elements[i].text)
                if i == len(menu_elements)-1: # 메뉴 리스트 길이면 나온나
                    break
                i+=1
            else:
                right_btn.click()
                sleep(SCROLL_PAUSE_SEC)
                driver.implicitly_wait(WAIT)
    except:
        pass
            
    #last_scroll = driver.execute_script("return document.body.scrollHeight")
    view_more_count = 0
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.5)
        if view_more_count ==19:
            break
        try :
            driver.implicitly_wait(3)
            #더보기 버튼 클릭
            driver.find_element(By.CSS_SELECTOR,'a.fvwqf').click()
            view_more_count += 1
            
        except:
            #더보기 버튼이 없다면 while문을 빠져나온다.
            break
            # 끝까지 스크롤 다운
            
        #new_scroll = driver.execute_script("return document.body.scrollHeight")
        #driver.implicitly_wait(WAIT)
        #if last_scroll == new_scroll:
        #    break
        #else:
        #    last_scroll= new_scroll

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    #가게 리스트 마다 방문자 리뷰 가져오기
    review_elements = driver.find_elements(By.CSS_SELECTOR,"div.ZZ4OK > a > span")
    review_list = []
    for i in review_elements:
        review_list.append(i.text)
    
    return menu_list, review_list


# 블로그 리뷰 크롤링 함수
## 불필요한 코퍼스가 많다.

# 검색 지역 횟집의 리뷰를 크롤링.
## 한번에 하는 코드 / refresh()를 해도 응답이 없음

In [44]:
def crawling(driver, i, store_elements,review_dic):
    driver.implicitly_wait(1)
    store_elements[i].click() #li의 스토어를 클릭
    driver.implicitly_wait(WAIT) # 2초 휴식
    try:
        #다른 iframe으로 변경
        switch_frame('entryIframe',driver)
        time_wait(2, '.Fc1rA',driver)

        # -----매장명 가져오기-----
        store_name = driver.find_element(By.CSS_SELECTOR,'.Fc1rA').text
        driver.implicitly_wait(2)
        review_dic[store_name] = {}
        
        # -----평점 가져오기-----
        try :
            rank = driver.find_element(By.CSS_SELECTOR,'div.zD5Nm.f7aZ0 > div.dAsGb > span.PXMot.LXIwF > em')
            driver.implicitly_wait(WAIT)
            review_dic[store_name]['rank'] = rank.text
        except:
            pass

        # -----위치 가져오기-----
        try : 
            location = driver.find_element(By.CSS_SELECTOR, 'span.IH7VW').text
            driver.implicitly_wait(WAIT)
            review_dic[store_name]['location'] = location
        except:
            pass

        #menu_list
        memu_list, review_list = crawling_menu_review(driver)
        
        # 메뉴 크롤링
        review_dic[store_name]['menu'] = menu_list
        
        # 방문자 리뷰 크롤링
        review_dic[store_name]['review'] = review_list
        driver.implicitly_wait(WAIT)
    except:
        pass
    

## 3번에 걸쳐 1페이지를 크롤링

In [45]:
def review_crawling(key_word, page_num):
    review_dic = dict()
    page_num = page_num-1
    for idx in range(3):
        url = 'https://map.naver.com/v5/search'
        driver = webdriver.Chrome('./chromedriver')
        driver.get(url)
        driver.implicitly_wait(2)
        
        # 검색어 입력
        find_search_input(key_word,driver=driver)
        driver.implicitly_wait(2)
        
        #프레임 변경
        switch_frame('searchIframe',driver)
        driver.implicitly_wait(WAIT)
        
        # 페이지 버튼 찾기
        pages = driver.find_elements(By.CSS_SELECTOR,'a.mBN2s')
        if page_num <= 5:
            pages[page_num].click()
            driver.implicitly_wait(WAIT)
        else:
            driver.find_elements(By.CSS_SELECTOR,'div.zRM9F > a:nth-child(6)').click()
            driver.implicitly_wait(WAIT)
            driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
            driver.implicitly_wait(WAIT)
            
        driver.implicitly_wait(WAIT)
        page_down(40,driver)
        sleep(1)
        driver.implicitly_wait(1)
        page_down(5,driver)
        sleep(2)
        driver.execute_script("window.scrollTo(0, 0);")
        driver.implicitly_wait(WAIT)
        store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
        print('total store_list : ', len(store_elements))
        
        
        #체크포인트
        ch_pt1 = len(store_elements) // 3
        ch_pt2 = ch_pt1*2
        if idx == 0:
            for i in range(ch_pt1):
                crawling(driver, i,store_elements,review_dic)
                switch_frame('searchIframe',driver)
                if i == ch_pt1-1:
                    print('first crawling store_list', ch_pt1)
                
                    
                    
                    
        elif idx == 1:
            for i in range(ch_pt1,ch_pt2):
                crawling(driver, i,store_elements,review_dic)
                switch_frame('searchIframe',driver)
                if i == ch_pt2-1:
                    print('second crawling store_list', ch_pt2-ch_pt1)
            
        else :
            for i in range(ch_pt2,len(store_elements)):
                crawling(driver, i,store_elements,review_dic)
                switch_frame('searchIframe',driver)
                if i == len(store_elements)-1:
                    print('third crawling store_list', len(store_elements)-ch_pt2)
                
                    
    # json 파일로 저장
    with open(f'naver_{key_word}_review_dic_page{page_num+1}.json', 'w', encoding='utf-8') as f:
        json.dump(review_dic, f, indent=4, ensure_ascii=False)
        
            
        
        
    
    
    

In [46]:
review_crawling('인천 횟집',1)

C:\Users\JSWonner\AppData\Local\Temp\ipykernel_12700\940980188.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  53
first crawling store_list 17
total store_list :  53
second crawling store_list 17
total store_list :  53
third crawling store_list 19


In [47]:
key_word_list = ['서울 횟집', '부산 횟집', '대구 횟집', ' 대전 횟집', '울산 횟집', '광주 횟집']

In [48]:
for key_word in key_word_list:
    review_crawling(key_word,1)

C:\Users\JSWonner\AppData\Local\Temp\ipykernel_12700\940980188.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  54
first crawling store_list 18
total store_list :  54
second crawling store_list 18
total store_list :  54
third crawling store_list 18
total store_list :  54
first crawling store_list 18
total store_list :  54
second crawling store_list 18
total store_list :  54
third crawling store_list 18
total store_list :  54
.Fc1rA 태그를 찾지 못하였습니다.


MaxRetryError: HTTPConnectionPool(host='localhost', port=12345): Max retries exceeded with url: /session/745769106dbcc26fbd7496623f5620a3/frame (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002E14856FC70>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [ ]:
with open(f'naver_{key_word}_review_dic_page0.json',encoding='utf-8') as f:
    naver_서울_횟집_review_dic_page0 = json.load(f)
with open(f'naver_{key_word}_review_dic_page1.json',encoding='utf-8') as f:
    naver_서울_횟집_review_dic_page1 = json.load(f)
with open(f'naver_{key_word}_review_dic_page2.json',encoding='utf-8') as f:
    naver_서울_횟집_review_dic_page2 = json.load(f)

In [ ]:
print(len(naver_서울_횟집_review_dic_page0.keys()))
print(len(naver_서울_횟집_review_dic_page1.keys()))
print(len(naver_서울_횟집_review_dic_page2.keys()))

In [ ]:
naver_서울_횟집_review_dic_page2

In [ ]:
review_dic.keys()

In [ ]:
review_dic

In [ ]:
range(10:20)

In [150]:
with open(f'naver_인천 횟집_review_dic_page1.json',encoding='utf-8') as f:
    naver_인천_횟집_review_dic_page0 = json.load(f)

In [151]:
naver_인천_횟집_review_dic_page0

{'대신수산 영종구읍뱃터점': {'rank': '4.79',
  'location': '인천 남동구 구월로 323 2층',
  'menu': [],
  'review': ['회가 싱싱하고 코스구성으로 회, 초밥, 다양한 스키 다시, 매운탕 , 튀김 까지 나왔는데 가격이 저렴해요 매장도 넓고 깨끗하고 수조에도 광어우럭이 ...',
   '내용 더보기',
   '연인세트먹었는데 회 구성이 좋았어요 양도 둘이먹 기에 많은편인데 4만원에 어디가서 이렇게 푸짐하게 먹을수없을텐데 가성비 최고네요 매운탕도 ...',
   '내용 더보기',
   '회 진짜 탱글쫄깃해요! 특히 튀김 너무 너무 맛있어요ㅠㅠ 튀김옷은 얇고 바삭하고 기름 쩐 맛 하나 없고 너무너무 맛있었어요! 매운탕 알밥도 강추♥',
   '정말 신선하구 매장도 넓고 맛나용^^ \n기본으로 나오는것들이 딱기본인데도 맛있고 \n친절하셔서 좋네용 다음에또 방문예정!',
   '둘이 가서 연인세트 시켰는데 셋이 먹어도 충분할 정 도로 양이 푸짐했어요 회도 세종류로 푸짐하게 나왔 구요 반찬은 모두 맛있었는데 특히 매운탕이 ...',
   '내용 더보기',
   '와 일단 가성비가 정말 좋아요!! 회도 완전 신선하고 \n소스도 다양하게 제공되서 좋았어요. 막장? 이 진짜 \n너무 맛있더라구요 👍🏻👍🏻 매운탕도 진짜 너무 ...',
   '내용 더보기',
   '매번 배민으로만 시켜먹다 이번엔 첨으로 포장주문 해서 방문했습니다. 사장님 엄청 친절하시네용 😻 배달도 엄청 만족했었는데 포장도 역시나 채...',
   '내용 더보기',
   '가성비 훌륭합니다.\n친절하시고 회도 정말 신선해요!! \n김치부침개 셀프 코너 대박이네오 :)',
   '너무너무 맛있고 신선해요 회를 남기지 않고 다 먹은 적은 정말 오랜만인것같아요 매장도 넓고 깨끗해서 단체모임으로 다음에 예약한번 하려구요! 세...',
   '내용 더보기',
   '매장도 넓고 횟감도 도톰하구 신선해요! 셀프부침개 재밌구 넘 맛있어요! 단체모임도 좋을듯ㅎ',
   '

In [ ]:
naver_인천_횟집_review_dic_page0

In [41]:
review_crawling('인천 횟집',2)

C:\Users\DanielJeong\AppData\Local\Temp\ipykernel_16044\940980188.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  48
first crawling store_list 16
total store_list :  48


WebDriverException: Message: chrome not reachable
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x003F78B3+2193587]
	Ordinal0 [0x00390681+1771137]
	Ordinal0 [0x002A4070+802928]
	Ordinal0 [0x00299AB2+760498]
	Ordinal0 [0x0028CEAF+708271]
	Ordinal0 [0x00291142+725314]
	Ordinal0 [0x002955C9+742857]
	Ordinal0 [0x002A5500+808192]
	Ordinal0 [0x002FD2C2+1168066]
	Ordinal0 [0x002ED5C6+1103302]
	Ordinal0 [0x002C77E0+948192]
	Ordinal0 [0x002C86E6+952038]
	GetHandleVerifier [0x006A0CB2+2738370]
	GetHandleVerifier [0x006921B8+2678216]
	GetHandleVerifier [0x004817AA+512954]
	GetHandleVerifier [0x00480856+509030]
	Ordinal0 [0x0039743B+1799227]
	Ordinal0 [0x0039BB68+1817448]
	Ordinal0 [0x0039BC55+1817685]
	Ordinal0 [0x003A5230+1856048]
	BaseThreadInitThunk [0x75A6FA29+25]
	RtlGetAppContainerNamedObjectPath [0x774075F4+228]
	RtlGetAppContainerNamedObjectPath [0x774075C4+180]


In [11]:
#수산물 데이터에 따른 사람 선호도에 따른 긍정, 부정 판단, 추천 프로젝트를 위한 데이터 크롤링

# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
#options = webdriver.ChromeOptions()
#options.add_argument('headless')
url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')
#driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)


C:\Users\DanielJeong\AppData\Local\Temp\ipykernel_18436\4021515187.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [12]:
find_search_input(key_word='인천 횟집',driver=driver)

In [97]:
switch_frame('searchIframe',driver)

In [14]:
page_down(40, driver)

In [43]:
store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')

In [98]:
store_elements[40].click()

In [99]:
switch_frame('entryIframe',driver)

In [100]:
find_review_btn(driver).click()

In [101]:
page_down(1, driver)

In [103]:
#try:
#메뉴 요소 찾기
menu_elements  = driver.find_elements(By.CSS_SELECTOR,'div.place_fixed_subtab > div > div:nth-child(1) > div > div > div > span > a> span:first-child')

try:
    #왼쪽 버튼 찾기
    left_btn = driver.find_element(By.CSS_SELECTOR,'a.PznE8.aNw43')
    left_btn.click()
except:
    pass

try:
    #오른족 버튼 찾기
    right_btn = driver.find_element(By.CSS_SELECTOR, 'div.place_fixed_subtab > div > div:nth-child(1) > div > div > a.ZCqf_.aNw43')
except:
    pass
#except:
    #pass
    
menu_list = list()
#-----------여기까지-----------
i = 0
# menu_elentents가 존재하지 않는다면 except로 갈 것이다.
try: 
    while True:
        if menu_elements[i].text != '': #요소가 존재하고 메뉴 텍스트도 보일때
            print(menu_elements[i].text)
            menu_list.append(menu_elements[i].text)
            if i == len(menu_elements)-1: # 메뉴 리스트 길이면 나온나
                break
            i+=1
        else:
            right_btn.click()
            sleep(2)
            driver.implicitly_wait(WAIT)
except:
    pass

물회
새우
튀김
새우튀김
매운탕
초밥
서더리탕
회덮밥
주먹밥
회무침
해물탕
밴댕이
날치알주먹밥
멍게
광어


In [140]:
len(menu_elements)

15

In [133]:
for i in range(len(menu_elements)):
    print(menu_elements[i].text)

물회
새우
튀김
새우튀김













In [134]:
menu_elements[4].text

''

In [89]:
del left_btn

In [91]:
menu_list

['매운탕', '연어', '낙지', '치즈', '콘치즈']

In [92]:
right_btn.click()

In [94]:
left_btn.click()

In [ ]:
def crawling_menu_review(driver):
    #-----리뷰 버튼 클릭-------
    find_review_btn(driver).click()
    SCROLL_PAUSE_SEC = 1
    page_down(1, driver)
    # ----메뉴 가져오기 ------
    menu_list = list()
    try:
        #메뉴 요소 찾기
        menu_elements  = driver.find_elements(By.CSS_SELECTOR,'div.place_fixed_subtab > div > div:nth-child(1) > div > div > div > span > a> span:first-child')
   
        try:
            #왼쪽 버튼 찾기
            left_btn = driver.find_element(By.CSS_SELECTOR,'a.PznE8.aNw43')
            left_btn.click()
        except:
            pass
        
        try:
            #오른족 버튼 찾기
            right_btn = driver.find_element(By.CSS_SELECTOR, 'div.place_fixed_subtab > div > div:nth-child(1) > div > div > a.ZCqf_.aNw43')
        except:
            pass

        i = 0
        while True: 
            #창에서 보이고, 겹치지 않는다면
            if menu_elements[i].text != '': # 만약 null은 생각안해도 된다.
                menu_list.append(menu_elements[i].text)
                i+=1
                if i == len(menu_elements):
                    break
            else : ## ''이라면
                right_btn().click()
            driver.implicitly_wait(3)
    except:
        pass
            